## Linear RegressionTask_2


#### Data Set Link: https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset

In [1]:
#important imports

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from pandas_profiling import ProfileReport

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso ,RidgeCV,LassoCV , ElasticNet , ElasticNetCV,LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [42]:
#loading the data
df = pd.read_csv("ai4i2020.csv")

In [39]:
#head command to show first five record
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [4]:
#profile report from pandas_profiling

pf = ProfileReport(df)
pf.to_widgets()

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
#saving report to html file
pf.to_file('lrtask.html')

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
#dropping UDI, Product ID and Type

df = df.drop("UDI", axis = 1)
df = df.drop("Product ID", axis = 1)

In [33]:
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [8]:
#correlation of Air Temperature with other variables
df.corr()["Air temperature [K]"]

Air temperature [K]        1.000000
Process temperature [K]    0.876107
Rotational speed [rpm]     0.022670
Torque [Nm]               -0.013778
Tool wear [min]            0.013853
Machine failure            0.082556
TWF                        0.009955
HDF                        0.137831
PWF                        0.003470
OSF                        0.001988
RNF                        0.017688
Name: Air temperature [K], dtype: float64

In [41]:
df.columns

Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [44]:
#Changing columns name

nw_col_names = ['Type', 'Air_temperature', 'Process_temperature','Rotational_speed', 'Torque', 'Tool_wear',
'Machine_failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']

df.columns = nw_col_names

### Statistical model for feature selection

In [11]:
import statsmodels.formula.api as smf
lm = smf.ols(formula='Air_temperature ~ Process_temperature +Rotational_speed +Torque +Tool_wear +Machine_failure +TWF +HDF +PWF +OSF +RNF', data=df).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Air_temperature   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     3454.
Date:                Fri, 03 Sep 2021   Prob (F-statistic):               0.00
Time:                        20:54:38   Log-Likelihood:                -13649.
No. Observations:               10000   AIC:                         2.732e+04
Df Residuals:                    9989   BIC:                         2.740e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -64.2001      2.000    -32.107      0.000     -68.120     -60.281
Process_temperature     1.1738      0.006    183.284      0.000       1.161       1.186
Rotational_speed        0.0002      0.000      1.582      0.114   -4.55e-05       0.000
Torque                  0.0003      0.002      0.123      0.902      -0.004       0.005
Tool_wear            8.455e-05      0.000      0.557      0.578      -0.000       0.000
Machine_failure        -0.0901      0.180     -0.500      0.617      -0.443       0.263
TWF                     0.2018      0.218      0.923      0.356      -0.227       0.630
HDF                     1.7793      0.191      9.334      0.000       1.406       2.153
PWF                     0.1420      0.186      0.763      0.445      -0.223       0.507
OSF                    -0.0405      0.176     -0.230      0.818      -0.386       0.305
RNF                    -0.0572      0.218     -0.263      0.793      -0.484       0.370
==============================================================================
Omnibus:                      647.802   Durbin-Watson:                   0.074
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              241.320
Skew:                          -0.091   Prob(JB):                     3.96e-53
Kurtosis:                       2.261   Cond. No.                     3.34e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### From above stats modle we can see that more features have p value more than 5%.

- Rotational_speed
- Torque
- Tool_Wear
- Machine_Failure
- TWF
- PWF
- OSF
- RNF

##### but i am going to drop only Torque, Tool_Wear, TWF, OSF and RNF feature for further model building. Not dropping Rotational_speed,  Machine_Failure.

In [45]:
df.drop(columns = ["Torque", "Tool_wear", "TWF", "OSF", "RNF", "PWF"], inplace = True)

In [46]:
df.head()

,Type,Air_temperature,Process_temperature,Rotational_speed,Machine_failure,HDF
0,M,298.1,308.6,1551,0,0
1,L,298.2,308.7,1408,0,0
2,L,298.1,308.5,1498,0,0
3,L,298.2,308.6,1433,0,0
4,L,298.2,308.7,1408,0,0


In [47]:
#Dummy variable creation for variable Type
df = pd.get_dummies(data = df, columns = ["Type"], drop_first = True)

In [48]:
df.head()

,Air_temperature,Process_temperature,Rotational_speed,Machine_failure,HDF,Type_L,Type_M
0,298.1,308.6,1551,0,0,0,1
1,298.2,308.7,1408,0,0,1,0
2,298.1,308.5,1498,0,0,1,0
3,298.2,308.6,1433,0,0,1,0
4,298.2,308.7,1408,0,0,1,0


In [16]:
#correlation after dummy variable creation
df.corr()["Air_temperature"]

Air_temperature        1.000000
Process_temperature    0.876107
Rotational_speed       0.022670
Tool_wear              0.013853
Machine_failure        0.082556
TWF                    0.009955
HDF                    0.137831
PWF                    0.003470
Type_L                 0.006676
Type_M                 0.007958
Name: Air_temperature, dtype: float64

### VIF check for multicollinearity

In [49]:
x = df.drop("Air_temperature", axis = 1)
y = df["Air_temperature"]

In [50]:
x.head()

,Process_temperature,Rotational_speed,Machine_failure,HDF,Type_L,Type_M
0,308.6,1551,0,0,0,1
1,308.7,1408,0,0,1,0
2,308.5,1498,0,0,1,0
3,308.6,1433,0,0,1,0
4,308.7,1408,0,0,1,0


In [51]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [52]:
vif_data = pd.DataFrame()
vif_data["feature"] = x.columns

vif_data["VIF"] = [variance_inflation_factor(x.values, i)
                          for i in range(len(x.columns))]

In [53]:
vif_data

,feature,VIF
0,Process_temperature,84.951208
1,Rotational_speed,75.900475
2,Machine_failure,1.551916
3,HDF,1.535279
4,Type_L,6.990186
5,Type_M,3.989532


In [54]:
df

,Air_temperature,Process_temperature,Rotational_speed,Machine_failure,HDF,Type_L,Type_M
0,298.1,308.6,1551,0,0,0,1
1,298.2,308.7,1408,0,0,1,0
2,298.1,308.5,1498,0,0,1,0
3,298.2,308.6,1433,0,0,1,0
4,298.2,308.7,1408,0,0,1,0
...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,0,0,0,1
9996,298.9,308.4,1632,0,0,0,0
9997,299.0,308.6,1645,0,0,0,1
9998,299.0,308.7,1408,0,0,0,0


In [55]:
lm = smf.ols(formula='Air_temperature ~ Process_temperature +Rotational_speed + Machine_failure +HDF +Type_L +Type_M', data=df).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Air_temperature   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.776
Method:                 Least Squares   F-statistic:                     5758.
Date:                Fri, 03 Sep 2021   Prob (F-statistic):               0.00
Time:                        21:35:31   Log-Likelihood:                -13649.
No. Observations:               10000   AIC:                         2.731e+04
Df Residuals:                    9993   BIC:                         2.736e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -64.1569      1.984    -32.343      0.000     -68.045     -60.269
Process_temperature     1.1736      0.006    183.333      0.000       1.161       1.186
Rotational_speed        0.0002   5.33e-05      3.524      0.000    8.34e-05       0.000
Machine_failure        -0.0009      0.064     -0.014      0.989      -0.127       0.125
HDF                     1.6942      0.110     15.458      0.000       1.479       1.909
Type_L                  0.0391      0.032      1.207      0.227      -0.024       0.102
Type_M                  0.0491      0.035      1.421      0.155      -0.019       0.117
==============================================================================
Omnibus:                      645.304   Durbin-Watson:                   0.074
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              240.348
Skew:                          -0.089   Prob(JB):                     6.44e-53
Kurtosis:                       2.262   Cond. No.                     3.31e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.31e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Model Building 

In [59]:
x_train, x_test, y_train,y_test = train_test_split(x, y,test_size= 0.20,random_state = 100)

### Simple Linear Regression

In [60]:
lr = LinearRegression()

In [61]:
lr.fit(x_train,y_train)

LinearRegression()

In [62]:
lr.score(x_test,y_test)

0.7967909304657287

In [63]:
# Let's create a function to create adjusted R-Squared
def adj_r2(x,y):
    r2 = lr.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [64]:
adj_r2(x_test,y_test)

0.7961791620677329

### Regularized Linear Models

#### Lasso

In [66]:
lassocv = LassoCV(alphas=None,cv= 50 , max_iter=2000, normalize=True)
lassocv.fit(x_train,y_train)

LassoCV(cv=50, max_iter=2000, normalize=True)

In [67]:
lassocv.alpha_

2.7470445071576778e-05

In [68]:
lasso = Lasso(alpha=lassocv.alpha_)
lasso.fit(x_train,y_train)

Lasso(alpha=2.7470445071576778e-05)

In [69]:
lasso.score(x_test,y_test)

0.7967878517630747

#### Ridge

In [70]:
ridgecv = RidgeCV(alphas=np.random.uniform(0,10,50),cv = 10 , normalize=True)
ridgecv.fit(x_train,y_train)

RidgeCV(alphas=array([1.17426296, 9.35117543, 0.89734849, 3.24986862, 4.14702618,
       5.24728724, 4.22729436, 5.68247538, 0.27437964, 4.59139472,
       7.33685965, 1.1044388 , 6.03938252, 0.46552932, 2.62376162,
       1.13186224, 5.66255387, 9.13317261, 1.71555174, 1.9675469 ,
       3.83631803, 8.69956165, 3.03429415, 5.72306633, 0.32184306,
       3.4509634 , 1.93781685, 9.08737662, 5.56065874, 6.05048598,
       3.10245737, 3.4298182 , 5.63522737, 4.45181277, 6.67769167,
       5.80329099, 8.79222407, 7.39750702, 8.3777295 , 4.52286965,
       4.26083085, 0.42748532, 2.90608086, 1.81325063, 7.98781687,
       6.29534044, 1.5524078 , 7.77226738, 6.77852176, 3.03858203]),
        cv=10, normalize=True)

In [71]:
ridgecv.alpha_

0.27437964132334347

In [72]:
ridge_lr = Ridge(alpha=ridgecv.alpha_)
ridge_lr.fit(x_train,y_train)

Ridge(alpha=0.27437964132334347)

In [73]:
ridge_lr.score(x_test,y_test)

0.7967868958772809

#### Elastic Net

In [74]:
elastic= ElasticNetCV(alphas=None, cv = 10 )
elastic.fit(x_train,y_train)

ElasticNetCV(cv=10)

In [75]:
elastic.alpha_

0.011924026731250032

In [76]:
elastic.l1_ratio_

0.5

In [77]:
elastic_lr = ElasticNet(alpha=elastic.alpha_ , l1_ratio=elastic.l1_ratio_)

In [78]:
elastic_lr.fit(x_train,y_train)

ElasticNet(alpha=0.011924026731250032)

In [79]:
elastic_lr.score(x_test,y_test)

0.7940965282689258

### Model Saving

In [80]:
pickle.dump(lr,open('LR.pickle','wb'))

In [81]:
model = pickle.load(open('LR.pickle','rb'))

In [83]:
x.head()

,Process_temperature,Rotational_speed,Machine_failure,HDF,Type_L,Type_M
0,308.6,1551,0,0,0,1
1,308.7,1408,0,0,1,0
2,308.5,1498,0,0,1,0
3,308.6,1433,0,0,1,0
4,308.7,1408,0,0,1,0


In [87]:
model.predict([[308.6,1551,0,0,0,1]])

array([298.3610447])

In [85]:
y

0       298.1
1       298.2
2       298.1
3       298.2
4       298.2
        ...  
9995    298.8
9996    298.9
9997    299.0
9998    299.0
9999    299.0
Name: Air_temperature, Length: 10000, dtype: float64